Google doc for the plan

https://docs.google.com/document/d/1Blv7YcRjNe_QuOfNGaQCoR9JN8CkooW2xq5PipLgCno/edit

In [ ]:
!ls /datasets/facebook_data

facebook-recruiting-iii-keyword-extraction.zip


In [ ]:
from zipfile import ZipFile 
import pandas as pd

This extracted the zip file directly in our files

In [ ]:
#from zipfile import ZipFile 
  
# # specifying the zip file name 
#file_name = "/datasets/facebook_data/facebook-recruiting-iii-keyword-extraction.zip"
  
# # opening the zip file in READ mode 
# with ZipFile(file_name, 'r') as zip: 
#    # printing all the contents of the zip file 
#    zip.printdir() 
  
#    # extracting all the files 
#    print('Extracting all the files now...') 
#    zip.extractall() 
#    print('Done!') 

File Name                                             Modified             Size
SampleSubmission.csv                           2019-12-11 23:27:44     82546830
Test.zip                                       2019-12-11 23:27:54    760327055
Train.zip                                      2019-12-11 23:29:10   2347110159
Extracting all the files now...
Done!


TRAIN is too big to into a pandas frame :(

In [ ]:
import pickle
import pandas as pd
import numpy as np
import re
# import warnings
# warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from sklearn.model_selection import train_test_split
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn import metrics
# from sklearn.metrics import f1_score,precision_score,recall_score
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_set = pickle.load(open("shorter_train_set.pickle", "rb"))

In [ ]:
train_set.head()

,Unnamed: 0,Id,Title,Body,Tags
0,0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


In [ ]:
train_set = train_set.drop_duplicates(['Title', 'Body', 'Tags'])
train_set["tag_count"] = train_set["Tags"].apply(lambda row : len(str(row).split(" ")))

In [ ]:
train_set["tag_count"] = train_set["Tags"].apply(lambda x : len(x.split()))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

Code and weird character in the text, we take that off

In [ ]:
qus_list = []
qus_with_code = 0
len_before_preprocessing = 0 
len_after_preprocessing = 0 
for index,row in train_set.iterrows():
    title, body, tags = row["Title"], row["Body"], row["Tags"]
    if '<code>' in body:
        qus_with_code+=1
    len_before_preprocessing+=len(title) + len(body)
    body=re.sub('<code>(.*?)</code>', '', body, flags=re.MULTILINE|re.DOTALL)
    body = re.sub('<.*?>', ' ', str(body.encode('utf-8')))
    title=title.encode('utf-8')
    question=str(title)+" "+str(body)
    question=re.sub(r'[^A-Za-z]+',' ',question)
    words=word_tokenize(str(question.lower()))
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    qus_list.append(question)
    len_after_preprocessing += len(question)
train_set["question"] = qus_list
avg_len_before_preprocessing=(len_before_preprocessing*1.0)/train_set.shape[0]
avg_len_after_preprocessing=(len_after_preprocessing*1.0)/train_set.shape[0]
print( "Avg. length of questions(Title+Body) before preprocessing: ", avg_len_before_preprocessing)
print( "Avg. length of questions(Title+Body) after preprocessing: ", avg_len_after_preprocessing)
print ("% of questions containing code: ", (qus_with_code*100.0)/train_set.shape[0])

Avg. length of questions(Title+Body) before preprocessing:  1150.20596
Avg. length of questions(Title+Body) after preprocessing:  327.51725
% of questions containing code:  56.084


In [ ]:
train_set[['question','Tags']]

,question,Tags
0,check upload file imag without mime type like ...,php image-processing file-upload upload mime-t...
1,prevent firefox close press ctrl favorit edito...,firefox
2,error invalid type list variabl import matlab ...,r matlab machine-learning
3,replac special charact url probabl simpl simpl...,c# url encoding
4,modifi whoi contact detail use modifi function...,php api file-get-contents
...,...,...
99995,script rotat png photoshop follow photoshop ex...,photoshop extendscript jsx photoshop-script
99996,debug browser crash run silverlight app team t...,silverlight debugging internet-explorer crash
99997,problem utf italian russian charact use rss fe...,errors rss encoding
99998,issu fb like button face issu fb like button u...,facebook like


In [ ]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
y_multilabel = vectorizer.fit_transform(train_set['Tags'])

In [ ]:
vectorizer.get_feature_names()

['.a',
 '.bash-profile',
 '.class-file',
 '.each',
 '.emf',
 '.hgtags',
 '.htaccess',
 '.htpasswd',
 '.mov',
 '.net',
 '.net-1.1',
 '.net-2.0',
 '.net-3.5',
 '.net-4.0',
 '.net-4.5',
 '.net-assembly',
 '.net-cf-3.5',
 '.net-framework',
 '.net-install',
 '.net-micro-framework',
 '.net-remoting',
 '.net-security',
 '.net2.0',
 '.net3.5',
 '.net4.0',
 '.nettiers',
 '.obj',
 '.post',
 '.rdlc',
 '.refresh',
 '.sdf',
 '.war',
 '.wav',
 '.x',
 '1080p',
 '10gbethernet',
 '10gige',
 '16bit',
 '1and1',
 '2-legged',
 '2-tier',
 '2003',
 '2007',
 '2010',
 '2013',
 '24bit',
 '24hour',
 '2d',
 '2d-array',
 '2d-games',
 '2wire',
 '3-tier',
 '3.0',
 '3.0.1',
 '3.1',
 '301',
 '301-redirect',
 '32-bit',
 '32-vs-64-bit',
 '3270',
 '32bit',
 '32bit-64bit',
 '37-signals',
 '3750',
 '3d',
 '3d-engine',
 '3d-graphics',
 '3d-model',
 '3d-modelling',
 '3d-reconstruction',
 '3des',
 '3dmark',
 '3dsmax',
 '3g',
 '3gp',
 '3nf',
 '3rd-party-library',
 '3ware',
 '401',
 '403',
 '404',
 '404-error',
 '4d',
 '5.1',
 

In [ ]:
train_set['Tags']

0        php image-processing file-upload upload mime-t...
1                                                  firefox
2                                r matlab machine-learning
3                                          c# url encoding
4                                php api file-get-contents
                               ...                        
99995          photoshop extendscript jsx photoshop-script
99996        silverlight debugging internet-explorer crash
99997                                  errors rss encoding
99998                                        facebook like
99999             serialization asp.net-web-api iqueryable
Name: Tags, Length: 100000, dtype: object

In [ ]:
y_multilabel.shape

(100000, 18487)

In [ ]:
def tags_to_consider(n):
    tag_i_sum = y_multilabel.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=y_multilabel[:,sorted_tags_i[:n]]
    return yn_multilabel

def questions_covered_fn(numb):
    yn_multilabel = tags_to_consider(numb)
    x= yn_multilabel.sum(axis=1)
    return (np.count_nonzero(x==0))

In [ ]:
questions_covered = []
total_tags = y_multilabel.shape[1]
total_qus  = preprocessed_df.shape[0]
for i in range(100, total_tags, 100):
    questions_covered.append(np.round(((total_qus-questions_covered_fn(i))/total_qus)*100,3))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=902703f2-430f-48f3-ba3f-6c2fee66cf11' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>